# pyphot demonstration

In [49]:
"""Load necessary packages"""
import pyphot #synthetic photometry
from astropy.table import Table #synthetic photometry
import numpy as np #input/output
from astropy.io import fits #input/output
from matplotlib import pyplot as plt #visualisation

### pyphot defines two classes: Filter and Library. Each instance of the Filter class contains transmission information for one filter, and each instance of the Library class consists of a collection of Filter classes.

### Libraries are stored in the form of HDF5 files.

#### Load/view information from a library

In [50]:
"""Load entire HDF5 library"""
libraryName='/Users/sundar/work/SAGE/1708/dustfitter/synphot_PhIReSSTARTer.hd5'
filterLibrary = pyphot.get_library(fname=libraryName)

In [51]:
"""Names of the first ten filters available in the library"""
filterNames = filterLibrary.get_library_content()
for i in range(9):
    print filterNames[i]

HST_WFPC2_F791W
HST_WFC3_F621M
HST_WFC3_F125W
ans_3300
steward_h
HST_ACS_WFC_F606W
steward_j
steward_k
HST_WFC3_F680N


#### Load/view information for a (set of) filter(s)

In [52]:
"""Load information for a single filter"""
ans_3300 = filterLibrary.load_filters(['ans_3300']) #result is a list, even for a single filter!
"""Load information for a list of filters"""
filters = filterLibrary.load_filters(['ans_3300', 'steward_k'])

In [53]:
"""View filter information"""
ans_3300[0].info()

Filter object information:
    name:                 ans_3300
    detector type:        photon
    wavelength units:     AA
    central wavelength:   3292.836737 angstrom
    pivot wavelength:     3292.709202 angstrom
    effective wavelength: 3292.623065 angstrom
    norm:                 36.965000
    definition contains 23 points

    Zeropoints
        Vega: 21.196181 mag,
              3.32297779849e-09 erg / angstrom * centimeter ** 2 * second,
              1201.68535663 Jy
          AB: 19.995715 mag,
              1.00395430898e-08 erg / angstrom * centimeter ** 2 * second,
              3630.59058769 Jy
          ST: 21.100000 mag,
              3.6307805477e-09 erg / angstrom * centimeter ** 2 * second,
              1312.99577725 Jy
        


#### The raw information required to define a filter is the wavelength (WITH UNITS!), transmission, and the detector type (i.e., whether it is a "photon" or "energy" counter). All the other infomation above is automatically computed for each filter.

### Users can define their own filters using the Filter class, and use them either temporarily or store them permanently in a Library class.

# Computing synthetic photometry using pyphot

### Example 1: computing synthetic photometry for a single spectrum (Source: IRC+10216, instrument: ISO-SWS).
#### (Uses the extractPhotometry method available in helpers.py)

In [54]:
"""filters in which synthetic photometry is desired"""
filterNames = ['MCPS_U', 'MCPS_B', 'MCPS_U', 'MCPS_I', '2MASS_J', '2MASS_H', '2MASS_Ks',
                   'SPITZER_IRAC_36', 'SPITZER_IRAC_45', 'SPITZER_IRAC_58', 'SPITZER_IRAC_80',
                   'SPITZER_MIPS_24']

In [55]:
spec = np.loadtxt('/Users/sundar/work/SAGE/1708/dustfitter/demo/IRC+10216_ISO_SWS.dat', comments='#', usecols=(0,1))

In [56]:
libraryName = '/Users/sundar/work/SAGe/1708/dustfitter/synphot_PhIReSSTARTer.hd5'
#Retrieve filter responses interpolated over the wavelength grid
filterList = foo.loadFilters(filterNames, spec[:,0]*pyphot.unit['micron'], libraryName=libraryName)

In [57]:
#Upon execution, cls contains the central wavelengths for each filter, and sed the
#   synthetic photometry in that filter.
cls, sed = pyphot.extractPhotometry(spec[:,0], spec[:,1], filterList, Fnu=True, absFlux=False)
lamCen = np.array([a.magnitude for a in cls])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(spec[:,0], spec[:,1], '--', label='ISO SWS spectrum')
ax.plot(lamCen, sed, 'o', color='red', label='Synthetic photometry')
ax.set_xlabel('Wavelength (micron)')
ax.set_ylabel('Flux (Jy)', rotation=90.)
for i in range(len(filterNames)):
    ax.text(lamCen[i], sed[i]*1.2, filterNames[i], horizontalalignment='center', verticalalignment='bottom', fontsize=8, rotation=90.)
plt.title('Example 1: IRC+10216')
plt.show()


Photometry:|----------------------| 0/12  0% [time: 00:00, eta: ?, ? iters/sec]

IndexError: list assignment index out of range